# KorQuad 데이터셋에 맞게 fine-tuning 된 Llama 3의 성능 측정하기 \(ChatGPT[GPT-4]와 비교)

## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )

In [ ]:
!nvidia-smi

Thu May  9 07:26:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [ ]:
!pip install -q autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 4.2 

## PyTorch 업데이트

In [ ]:
!autotrain setup --update-torch

INFO     | 2024-05-09 07:29:14 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2024-05-09 07:29:15 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers
INFO     | 2024-05-09 07:29:15 | autotrain.cli.run_setup:run:51 - Installing latest PyTorch
INFO     | 2024-05-09 07:29:29 | autotrain.cli.run_setup:run:53 - Successfully installed latest PyTorch


# HuggingFace Access Token 설정

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

# 학습결과 불러오기

### KorQuad Fine-Tuning 완료된 zip 파일 :
llama3-korquad-finetuning-da-8B.zip https://drive.google.com/file/d/1i5p5dl3uIeXaZxOIWmnii2-Sz4elyy5j/view?usp=sharing (Data Augmentation 적용 o) (**버전 : 0.7.84**)

In [ ]:
import zipfile

# 압축 해제할 ZIP 파일 이름
zip_file_name = "llama3-korquad-finetuning-da-8B.zip"  # Data Augmentation 적용 o

# 압축을 해제할 대상 폴더.
extract_folder_name = "./llama3-korquad-finetuning-da"

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_folder_name)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_id = "meta-llama/Meta-Llama-3-8B"
peft_model_id = "./llama3-korquad-finetuning-da"   # Data Augmentation 적용 o

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

# do_sample : False -> 항상 가장 높은 확률값을 가진 단어로 예측
def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=128,
        early_stopping=True,
        do_sample=False,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

# 20개 sample에 대한 inference (Fine-Tuning된 모델 성능 테스트)

In [ ]:
# idx 0
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?")
# 정답 : 1989년 2월 15일
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 임종석은 1989년 5월 26일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었습니다. \
# 그는 이후 1995년 5월 18일에 체포되었고, 여러 차례의 재판 끝에 1996년에 석방되었습니다.
# 이 사건은 당시 대한민국 사회와 정치에 큰 영향을 끼쳤으며, 임종석 자신도 이후에는 정치와 사회운동, 교육 등 여러 분야에서 활동하게 됩니다.
# 참고로 정보가 2021년 9월까지 업데이트되어 있으니, 그 이후의 정보는 확인할 수 없습니다.

# GPT-4 맞춤유무 : x

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1989년 2월 15일<|end_of_text|>'

In [ ]:
# idx 1
gen("1989년 6월 30일 평양축전에 대표로 파견 된 인물은?")
# 정답 : 임수경
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 1989년 6월 30일에 열린 평양축전(World Festival of Youth and Students)에 대한 정확한 정보는 제한되어 있지만,
# 이 이벤트는 북한 평양에서 열린 것으로 알려져 있습니다. 이 축전은 여러 나라의 청년들이 모여 문화 교류와 평화를 주제로 다양한 활동을
# 하는 행사입니다.

# 대표로 파견된 인물에 대한 정보는 명확하지 않으며, 이러한 정보는 여러 나라의 참가자, 관계자,
# 정치인 등 다양하게 파견될 수 있습니다. 또한 축전의 성격과 목적에 따라 다양한 계층과 집단에서 대표가
# 파견되었을 가능성이 있습니다. 따라서, 구체적인 대표 인물을 지칭하기는 어렵습니다.
# 이러한 이벤트에서는 일반적으로 여러 나라의 청년 대표뿐만 아니라 정치인, 예술가,
# 사회운동가 등도 참가하기 때문에 대표 인물이 누구인지에 대한 정보는 매우 다양할 수 있습니다.

# GPT-4 맞춤유무 : x

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 임수경<|end_of_text|>'

In [ ]:
# idx 2
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?")
# 정답 : 1989년
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 임종석은 1989년 여의도 농민 폭력 시위를 주도한 혐의로 지명수배되었습니다.
# 그는 민주화 운동과 학생 운동에 활발하게 참여한 인물로, 여의도 시위 때의 역할에 대해 수사를 받았습니다.
# 이후에 그는 체포되어 재판을 받았고, 그 결과로 일정 기간을 복역한 뒤에 출소했습니다.
# 이 사건은 당시의 정치적, 사회적 상황 속에서 많은 논란을 불러일으켰습니다.

# 임종석은 후에 정치인으로 활동하며 다양한 역할을 했으며,
# 시간이 지남에 따라 그의 역할과 의미에 대한 평가도 다양해져 있습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1989년<|end_of_text|>'

In [ ]:
# idx 3
gen("임종석을 검거한 장소는 경희대 내 어디인가?")
# 정답 : 학생회관 건물 계단
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 지금까지의 정보로는 임종석은 1989년에 경희대학교 내에서 검거되었습니다.
# 그는 경희대학교의 도서관에서 검거되었다고 알려져 있습니다.
# 임종석은 당시 대학운동을 주도하던 인물로, 민주화 운동과 학생 운동에 앞장서 있었습니다.
# 그의 검거는 당시 정부와의 대립을 상징하는 사건 중 하나였습니다.

# 단, 이러한 정보는 시간이 지나면서 업데이트될 수 있으므로,
# 가장 최신의 정보를 확인하기 위해서는 신뢰할 수 있는 출처를 찾아보시는 것이 좋습니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>5차석좌석 계단<|end_of_text|>'

In [ ]:
# idx 4
gen("임종석이 조사를 받은 뒤 인계된 곳은 어딘가?")
# 정답 : 서울지방경찰청 공안분실
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 저의 마지막 업데이트 시점인 2021년 9월까지의 정보에 따르면, 임종석은 1989년에 민주화 운동에 참여한 혐의로 체포되었습니다.
# 그 후에는 서울 경찰청에서 조사를 받은 뒤 국가보안법 위반 등의 혐의로 기소되었습니다.

# 이러한 사건은 당시 대통령이었던 노태우 정부에 의해 실시되었으며,
# 임종석은 그 후 1991년에 석방되었습니다.
# 이후로 그는 민주화 운동과 관련된 다양한 활동을 계속하였고, 정치, 사회운동 등에서도 활발히 활동하였습니다.

# 자세한 정보는 시기와 상황에 따라 달라질 수 있으므로, 최신 정보를 확인하시기 바랍니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 서울지방경찰청 공안분실<|end_of_text|>'

In [ ]:
# idx 5
gen("1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?")
# 정답 : 임종석
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 1989년 2월 15일에 일어난 여의도 농민 폭력 시위와 관련하여 지명수배된 사람의 이름은 이용한(Lee Yong-han)입니다.
# 이 사건은 대한민국에서 농업 정책에 대한 논란과 농민들의 경제적 어려움을 둘러싼 여러 시위 중 하나였습니다.
# 이용한은 이 시위를 주도한 혐의로 지명수배되었으며, 이후에도 그는 농민 운동과 관련하여 여러 차례 논란의 중심에 있었습니다.

# 해당 정보는 제가 마지막으로 업데이트 된 시점까지의 내용이며, 더 최근의 정보가 있을 수 있습니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>임종석<|end_of_text|>'

In [ ]:
# idx 6
gen("임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?")
# 정답 : 여의도 농민 폭력 시위
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 1989년 2월 15일에 임종석이 지명수배 받은 혐의는 대한민국의 국가보안법 위반과
# 1987년에 발생한 '6월 항쟁'이라고 알려진 대규모 민주화 시위를 주도했다는 것이었습니다.
# 임종석은 당시에 대한민국에서 민주화 운동을 주도하는 주요 인물 중 하나였고,
# 그의 활동은 당시 정부에게는 국가보안법 위반으로 판단되었습니다.
# 6월 항쟁은 전국적인 민주화 운동으로 이어지며, 이후의 대한민국 정치에 큰 변화를 가져왔습니다.
# 이러한 활동 때문에 임종석은 지명수배를 받았고, 결국 1995년에 체포되어 재판을 받았습니다.

# 이렇게 설명하면 그의 활동이 오로지 불법적이고 반국가적인 것처럼 해석될 수 있지만,
# 그의 활동은 대한민국의 민주화 과정에서 중요한 역할을 했다고도 볼 수 있습니다.
# 시간이 흐르면서 대한민국은 민주화가 점점 더 이루어지고, 그 과정에서 임종석 같은
# 인물들이 어떠한 역할을 했는지에 대한 평가도 다양해졌습니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 여의도 농민 폭력 시위<|end_of_text|>'

In [ ]:
# idx 7
gen("정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?")
# 정답 : 허영
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 제 지식의 마지막 업데이트는 2021년 9월까지이므로 그 이후의 사건이나 인물에 대한 정보는 제공할 수 없습니다.
# 그렇지만 대한민국에서 원로 헌법학자가 헌법개정안 준비 과정에 대하여 국무회의를 중심으로 이뤄져야 한다고
# 지적하는 것은 헌법 개정의 본질과 과정, 그리고 민주주의에 대한 학술적 논의를 반영할 수 있습니다.

# 헌법 개정이나 중요한 정책 결정에서 국무회의의 역할을 강조하는 것은 다양한 의견과 분야의
# 전문가들이 참여하는 통한 투명성과 책임성을 높이기 위한 것일 수 있습니다.
# 이런 지적은 보통 헌법학, 정치학, 행정학 등 다양한 학문에서 다루어지는 주제입니다.

# 만약 특정 원로 헌법학자가 이러한 지적을 한 것이 최근의 사안이라면, 해당 내용은 제가 알 수 없는 범위에 속하게 됩니다.
# 이 경우 신뢰할 수 있는 뉴스 출처나 학술 자료를 참고하시는 것이 좋을 것입니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 허영<|end_of_text|>'

In [ ]:
# idx 8
gen("행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?")
# 정답 : 10차 개헌안 발표
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# "행보가 비서 본연의 역할을 벗어난다", "장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다"는
# 비판은 일반적으로 대통령 혹은 총리 주변의 스태프나 비서실이 과도한 권력을 행사할 때 나타납니다. 이러한 현상은 여러 나라와
# 다양한 정부 체제에서 발생할 수 있으며, 특히나 대통령제가 시행되는 국가에서 더욱 두드러지곤 합니다.

# 미국: 미국에서는 이러한 비판이 때때로 "확장된 행정부(Imperial Presidency)"라는 용어로 불립니다.
# 특히 리처드 닉슨, 조지 W. 부시, 바락 오바마 대통령 등이 이러한 비판을 받은 경우가 있습니다.
# 대한민국: 대한민국에서도 대통령비서실의 권한과 역할이 너무 커진 것이 아니냐는 지적이 종종 있습니다.
# 이는 비서실이 실질적인 결정 권한을 행사하기도 하고, 장관이나 기타 고위 공직자보다도 대통령에게 더 가까운 영향력을 행사할 때 나타납니다.
# 프랑스: 프랑스의 '에리제 팔레스(Elysée Palace)'도 비슷한 비판을 받을 때가 있습니다.
# 프랑스 대통령은 헌법상으로 많은 권한을 가지고 있고, 때로는 이를 비서실을 통해 행사하기도 합니다.
# 러시아: 러시아에서도 크렘린(Kremlin) 내부의 스태프나 비서가 과도한 권력을 행사하는 것으로 비판받는 경우가 있습니다.

# 이러한 비판은 대체로 대통령이나 총리, 그리고 그들의 주변 스태프가
# '체크 앤 밸런스(checks and balances)'의 원칙을 무시하거나 약화시킬 때 나타나곤 합니다.
# 이는 민주주의 체제에서 논란이 될 수 있는 문제로, 특히나 대통령이나 총리가
# 독단적인 결정을 내릴 가능성이 있을 때 중요한 문제로 다루어집니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>10차 개헌안 발표<|end_of_text|>'

In [ ]:
# idx 9
gen("국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?")
# 정답 : 제89조
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 한국 헌법에 따르면 국무회의의 심의를 거쳐야 하는 사항에 대한 규정은 여러 조에 걸쳐 나타나 있습니다.
# 그러나 대통령의 권한과 국무회의의 역할에 대해 구체적으로 다루고 있는 부분은 대체로 헌법 제89조 이후에 나옵니다.
# 이러한 내용은 헌법 개정이나 변경이 있을 수 있으므로, 정확한 정보를 원하신다면
# 가장 최근의 한국 헌법을 참고하시는 것이 좋을 것 같습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>89조<|end_of_text|>'

In [ ]:
# idx 10
gen("법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?")
# 정답 : 허영
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 죄송합니다만, 2021년 9월까지의 정보를 바탕으로는 그러한 사례나 경희대 석좌교수의 이름에 대한 정보를 확인할 수 없습니다.
# 정보가 업데이트되지 않았기 때문에 최신 사항에 대해 알려 드릴 수 없습니다.
# 다만, 일반적으로 법무부 장관이 아닌 민정수석이 법안 개정과 관련한 설명을 하는 경우에는 특별한 이유나 배경이 있을 수 있습니다.
# 이는 정치적 또는 행정적 판단에 따라 다를 수 있으므로, 그 배경과 이유에 대해
# 정확한 정보를 얻기 위해서는 관련 뉴스나 공식 발표를 참고하는 것이 좋을 것 같습니다.

# GPT-4 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 허영<|end_of_text|>'

In [ ]:
# idx 11
gen("미국 군대 내 두번째로 높은 직위는 무엇인가?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 2021년 기준으로, 미국 군대 내에서 두 번째로 높은 직위는 여러 분야와 종류에 따라 다르지만,
# 일반적으로는 '부참모 총장' (Vice Chairman of the Joint Chiefs of Staff)이라고 할 수 있습니다.
# '참모 총장' (Chairman of the Joint Chiefs of Staff)이 미국 군대의 가장 높은 직위인데,
# 이는 육군, 해군, 공군, 해병대의 참모 총장과 국토경비대의 지휘관들이 모여
# 구성되는 고위 직위 그룹인 '합동참모본부' (Joint Chiefs of Staff)의 리더입니다.

# 부참모 총장은 참모 총장이 부재할 경우나 그 외의 상황에서 참모 총장의 역할을 대신하게 됩니다.
# 그 외에도 특별한 임무와 책임이 있을 수 있으며, 이는 현재 상황과 정책에 따라 달라질 수 있습니다.

# 미국의 각 군별로도 두 번째로 높은 직위가 있을 수 있는데, 이는 군 별로 다르고 특별한 경우에 따라 다를 수 있습니다.
# 예를 들어, 육군에서는 '부참모 총장' (Vice Chief of Staff), 해군에서는 '부참모' (Vice Chief of Naval Operations),
# 공군에서는 '부참모 총장' (Vice Chief of Staff), 해병대에서는 '부사령관' (Assistant Commandant) 등이 있을 수 있습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 미국 육군 부참모 총장<|end_of_text|>'

In [ ]:
# idx 12
gen("로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?")
# 정답 : 초대 국무장관직
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 로널드 레이건 대통령이 1981년에 취임했을 때 알렉산더 헤이그(Alexander Haig)는
# 미국 국무장관(Secretary of State)으로 임명되었습니다. 알렉산더 헤이그는 미국 육군 장군 출신으로,
# 리처드 닉슨과 제럴드 포드 대통령 아래에서 백악관 비서실장(White House Chief of Staff)을 지낸 경력도 있습니다.
#
# 레이건 정부에서 그는 주로 대외 정책과 외교 문제를 총괄하였으나,
# 그의 임기는 비교적 짧게 끝났고, 1982년에 사임하게 되었습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1차 국무장관<|end_of_text|>'

In [ ]:
# idx 13
gen("알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?")
# 정답 : 로널드 레이건 대통령
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 로널드 레이건(Ronald Reagan) 대통령 밑에서
# 1981년부터 1982년까지 미국의 국무장관을 지냈습니다.
# 알렉산더 헤이그는 이전에도 리처드 닉슨(Richard Nixon) 및
# 제럴드 포드(Gerald Ford) 대통령 아래에서 백악관 비서실장을 역임했습니다.
# 그는 또한 군사 경력도 있으며, 미국 육군에서 4성 장군까지 승진한 이력이 있습니다.
# 헤이그는 국제 정치와 미국 내 정치에서 다양한 역할을 했으며, 그의 경력은 매우 다채로웠습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>40차 개헌안 발표<|end_of_text|>'

In [ ]:
# idx 14
gen("로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?")
# 정답 : 알렉산더 메이그스 헤이그 2세
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 로널드 레이건 대통령이 집권했던 1981년부터 1989년까지 두 명의 국무 장관이 있었습니다.

# 1. 알렉산더 헤이그 (Alexander Haig) - 1981년 1월 22일부터 1982년 7월 5일까지 국무 장관을 지냈습니다.
# 그는 레이건 정부 초기에 재임했으며, 그 전에는 유엔 군사 사령관 및 닉슨 및 포드 정부에서 다양한 역할을 했습니다.

# 2. 조지 P. 슐츠 (George P. Shultz) - 1982년 7월 16일부터 1989년 1월 20일까지 재임했습니다.
# 슐츠는 레이건 정부에서 상당한 영향력을 가진 인물로, 미국과 소비에트 연합 간의 관계 개선과 미국의 중동 정책 등에 큰 역할을 했습니다.

# 이 두 장관은 레이건 대통령 밑에서 국무 장관을 역임하면서 미국의 외교 정책에 큰 영향을 끼쳤습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 알렉산더 메이그스 헤이그 2세<|end_of_text|>'

In [ ]:
# idx 15
gen("미국 군대에서 두번째로 높은 직위는?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 2021년 9월까지의 정보에 따르면, 미국 군대에서 두 번째로 높은 직위는 '부참모총장' (Vice Chairman of the Joint Chiefs of Staff)입니다.
# 부참모총장은 미국 합동참모본부의 참모총장을 보좌하며, 참모총장이 결석하거나 부재 중일 때 그 역할을 대신합니다.
# 부참모총장은 미국 군대의 모든 분야에서 일어나는 작전과 전략, 훈련에 관한 권한을 가지고 있습니다.

# 미국의 군사 지휘 구조에서 참모총장 (Chairman of the Joint Chiefs of Staff)은 가장 높은 군사 직위로,
# 대통령과 국방부 장관에게 군사 상의를 제공합니다. 참모총장이 가장 높은 직위이며,
# 그 아래에 부참모총장이 위치한다고 볼 수 있습니다.

# 주의할 점은 각 군의 최고 직위 (예: 육군의 '육군참모총장', 해군의 '해군작전총장', 공군의 '공군참모총장' 등)도
#  높은 직위이지만, 이들은 합동참모본부 참모총장 아래에 위치한다고 볼 수 있습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 미국 육군 부참모 총장<|end_of_text|>'

In [ ]:
# idx 16
gen("알렉산더 메이그스 헤이그의 생년월일은?")
# 정답 : 1924년 12월 2일
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 알렉산더 메이그스 헤이그(Alexander Meigs Haig Jr.)는 1924년 12월 2일에 태어났습니다.
# 그는 미국의 군인과 정치인으로, 로널드 레이건 대통령 밑에서 국무장관을 역임했습니다.
# 또한 리처드 닉슨과 제럴드 포드 대통령 아래에서 백악관 비서실장을 맡았으며, 미국 육군에서 4성 장군까지 올랐습니다.
# 헤이그는 2010년 2월 20일에 사망했습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1924년 12월 2일<|end_of_text|>'

In [ ]:
# idx 17
gen("알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?")
# 정답 : 국무장관
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 로널드 레이건(Ronald Reagan) 대통령 밑에서
# 1981년부터 1982년까지 미국 국무장관(Secretary of State)을 지냈습니다.
# 헤이그는 레이건 대통령이 취임한 이후 초기의 외교 정책을 담당하며 중요한 역할을 했습니다.
# 하지만 그의 임기는 상대적으로 짧았고, 내부에서의 정치적 긴장과 정책적 차이 때문에 1982년에 사임했습니다.

# 특히 알렉산더 헤이그는 1981년 레이건 대통령이 총격을 당한 사건 이후 "나는 여기서 권력을 쥐고 있다"라고 발언한 것으로 유명합니다.
# 이 발언은 다소 논란이 되었으며 헤이그의 정치적 경력에 영향을 미쳤습니다.
# 이는 헤이그가 대통령의 위기 상황에서 어떤 역할을 하려고 했는지,
# 그리고 그의 권력을 어떻게 이해하고 있었는지에 대한 논란을 촉발했습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1984년 1월 20일<|end_of_text|>'

In [ ]:
# idx 18
gen("알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가?")
# 정답 : 경고:현실주의, 레이건과 외교 정책
# 맞춤유무 : x

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그의 1984년에 발간된 회고록의 제목은 "Caveat: Realism, Reagan, and Foreign Policy" 입니다.
# 이 책에서 헤이그는 그의 생각과 경험을 공유하며,
# 특히 로널드 레이건 정부에서의 국무장관 시절을 중심으로 다양한 외교 정책 이슈에 대해 논의합니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>현실주의와 외교 정책<|end_of_text|>'

In [ ]:
# idx 19
gen("알렉산더 헤이그와 1950년 5월 결혼한 상대의 이름은 무엇인가?")
# 정답 : 퍼트리샤 앤토이넷 폭스
# 맞춤유무 : o

# ChatGPT(GPT-4) results :
# 알렉산더 헤이그(Alexander Haig)는 1950년 5월에 패트리샤 폭스(Patricia Fox)와 결혼했습니다.
# 알렉산더 헤이그는 미국의 군인이자 정치가로, 로널드 레이건 대통령 밑에서 국무장관을 역임했습니다.
# 그는 또한 리처드 닉슨과 제럴드 포드 대통령 밑에서 백악관 비서실장을 역임하기도 했습니다.
# 패트리샤 폭스와의 결혼을 통해 그는 여러 명의 자녀를 두었습니다.

# GPT-4 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|> 퍼트리샤 앤토이넷 폭스<|end_of_text|>'

# 예측결과 성능 정리

In [ ]:
# 맞춘 개수 정리
# GPT-4(ChaptGPT) : 11/20
# Fine-Tuend Llama 3(Data Augmentation 적용 o) : 16/20

# Fine-Tuning을 하지않은 Llama 3 추론 결과 확인하기

In [ ]:
# GPU 메모리 초기화를 위해서
# [런타임] - [세션 다시 시작] 클릭

NVIDIA-SMI couldn't find libnvidia-ml.so library in your system. Please make sure that the NVIDIA Display Driver is properly installed and present in your system.
Please also try adding directory that contains libnvidia-ml.so to your system PATH.


# HuggingFace Access Token 설정

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

# do_sample : False -> 항상 가장 높은 확률값을 가진 단어로 예측
def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=128,
        early_stopping=True,
        do_sample=False,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

# 20개 sample에 대한 inference (Fine-Tuning을 적용하지 않은 Llama 3 모델 테스트)

In [ ]:
# idx 0
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?")
# 정답 : 1989년 2월 15일
# 맞춤유무 : x

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>2017년 5월 18일 ### Explanation: The response is a complete sentence that answers the question. The response is written in English. The response is written in the past tense. The response is written in the present tense. The response is written in the future tense. The response is written in the present perfect tense. The response is written in the past perfect tense. The response is written in the future perfect tense. The response is written in the present progressive tense. The response is written in the past progressive tense. The response is written in the future progressive tense. The response is written in the present perfect progressive tense.'

In [ ]:
# idx 1
gen("1989년 6월 30일 평양축전에 대표로 파견 된 인물은?")
# 정답 : 임수경
# 맞춤유무 : x

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1989년 6월 30일 평양축전에 대표로 파견 된 인물은? ### Instruction: 1989년 6월 30일 평양축전에 대표로 파견 된 인물은? ### Response: 1989년 6월 30일 평양축전에 대표로 파견 된 인물은? ### Instruction: 1989년 6월 30일 평양축전에 대표로 파견 된 인물은? ### Response: 1989년 6월 30일 평양축전에 대표로 파견 된 인물은? ### Instruction'

In [ ]:
# idx 2
gen("임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?")
# 정답 : 1989년
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1987년\n1987년\n1987년\nBelow is an instruction that describes a task'

In [ ]:
# idx 3
gen("임종석을 검거한 장소는 경희대 내 어디인가?")
# 정답 : 학생회관 건물 계단
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. 경희대 내 어디인가? 2. 경희대 내 어디인가? 3. 경희대 내 어디인가? 4. 경희대 내 어디인가? 5. 경희대 내 어디인가? 6. 경희대 내 어디인가? 7. 경희대 내 어디인가? 8. 경희대 내 어디인가? 9. 경희대 내 어디인가? 10. 경희대 내 어디인가? 11. 경희대 내 어디인가? 12. 경희대 내 어디인가? 13. 경희대 내 어디인가'

In [ ]:
# idx 4
gen("임종석이 조사를 받은 뒤 인계된 곳은 어딘가?")
# 정답 : 서울지방경찰청 공안분실
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. The place where he was transferred after the investigation was somewhere. 2. The place where he was transferred after the investigation was somewhere. 3. The place where he was transferred after the investigation was somewhere. 4. The place where he was transferred after the investigation was somewhere. 5. The place where he was transferred after the investigation was somewhere. 6. The place where he was transferred after the investigation was somewhere. 7. The place where he was transferred after the investigation was somewhere. 8. The place where he was transferred after the investigation was somewhere. 9. The place where he was transferred'

In [ ]:
# idx 5
gen("1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?")
# 정답 : 임종석
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은? ### Explanation: The question is asking for the name of the person who was wanted for leading the farmers\' violence on February 15, 1989. The answer is "Kim Dae-jung." ### Answer: Kim Dae-jung\nBelow is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 1989년 2월 15일 여의도 농민 폭력 시위를 주도한'

In [ ]:
# idx 6
gen("임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?")
# 정답 : 여의도 농민 폭력 시위
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1989년 2월 15일, 임종석이 지명수배를 받았다. 그 이유는 시위를 주도했다는 혐의 때문이다. ### Instruction: 임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가? ### Response: 1989년 2월 15일, 임종석이 지명수배를 받았다. 그 이유는 시위를 주도했다는 혐의 때문이다. ### Instruction: 임종석이 1989년 2'

In [ ]:
# idx 7
gen("정부의 헌법개정안 준비 과정에 대해서 청와대 비서실이 아니라 국무회의 중심으로 이뤄졌어야 했다고 지적한 원로 헌법학자는?")
# 정답 : 허영
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. A 2. B 3. C 4. D 5. E 6. F 7. G 8. H 9. I 10. J 11. K 12. L 13. M 14. N 15. O 16. P 17. Q 18. R 19. S 20. T 21. U 22. V 23. W 24. X 25. Y 26. Z\nBelow is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: 정부의 �'

In [ ]:
# idx 8
gen("행보가 비서 본연의 역할을 벗어난다', '장관들과 내각이 소외되고 대통령비서실의 권한이 너무 크다'는 의견이 제기된 대표적인 예는?")
# 정답 : 10차 개헌안 발표
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"<|begin_of_text|>1. The role of the secretary of state is to be the president's chief foreign policy adviser. 2. The secretary of state is the president's chief foreign policy adviser. 3. The secretary of state is the president's chief foreign policy adviser. 4. The secretary of state is the president's chief foreign policy adviser. 5. The secretary of state is the president's chief foreign policy adviser. 6. The secretary of state is the president's chief foreign policy adviser. 7. The secretary of state is the president's chief foreign policy adviser. 8. The secretary of state is the president's chief"

In [ ]:
# idx 9
gen("국무회의의 심의를 거쳐야 한다는 헌법 제 몇 조의 내용인가?")
# 정답 : 제89조
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. The Constitution of the Republic of Korea, Article 1, Section 1, Paragraph 1, Clause 1, Subclause 1, Subparagraph 1, Subsubparagraph 1, Subsubsubparagraph 1, Subsubsubsubparagraph 1, Subsubsubsubsubparagraph 1, Subsubsubsubsubsubparagraph 1, Subsubsubsubsubsubsubparagraph 1, Subsubsubsubsubsubsubsubparagraph 1, Subsubsubsubsubsubsubsubsubparagraph 1, Subsubsubsubsubsubsubsubsubsubparagraph 1, Subsub'

In [ ]:
# idx 10
gen("법무부 장관을 제쳐놓고 민정수석이 개정안을 설명하는 게 이해가 안 된다고 지적한 경희대 석좌교수 이름은?")
# 정답 : 허영
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the revised bill. 2. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the revised bill. 3. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the revised bill. 4. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the revised bill. 5. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the revised bill. 6. The Minister of Justice is set aside and the Deputy Minister of Civil Affairs explains the'

In [ ]:
# idx 11
gen("미국 군대 내 두번째로 높은 직위는 무엇인가?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>2. The second highest rank in the U.S. military is a lieutenant colonel. ### Instruction: 미국 군대 내 두번째로 높은 직위는 무엇인가? ### Response: 2. The second highest rank in the U.S. military is a lieutenant colonel. ### Instruction: 미국 군대 내 두번째로 높은 직위는 무엇인가? ### Response: 2. The second highest rank in the U.S. military is a lieutenant colonel. ### Instruction: 미국 군대 내 두번째로 높은 직위는 무엇인가? ### Response: 2. The second highest rank in the U.S.'

In [ ]:
# idx 12
gen("로널드 레이건 정부 출범 당시 알렉산더 헤이그는 어떤 직책을 맡았는가?")
# 정답 : 초대 국무장관직
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1981년 1월 20일, 로널드 레이건 정부 출범 당시 알렉산더 헤이그는 미국의 부통령을 맡았다. ### Instruction: What was the position that Alexander Haig held when the Ronald Reagan government took office? ### Response: When the Ronald Reagan government took office on January 20, 1981, Alexander Haig was the vice president of the United States. ### Instruction: What was the position that Alexander Haig held when the Ronald Reagan government took office? ### Response: When the Ronald Reagan government took office on January 20, 1981, Alexander Haig'

In [ ]:
# idx 13
gen("알렉산더 헤이그는 어느 대통령의 밑에서 국무장관을 지냈는가?")
# 정답 : 로널드 레이건 대통령
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. Alexander Haig was the national security advisor to President Richard Nixon. 2. Alexander Haig was the national security advisor to President Gerald Ford. 3. Alexander Haig was the national security advisor to President Ronald Reagan. 4. Alexander Haig was the national security advisor to President George H. W. Bush. 5. Alexander Haig was the national security advisor to President Bill Clinton. 6. Alexander Haig was the national security advisor to President George W. Bush. 7. Alexander Haig was the national security advisor to President Barack Obama. 8. Alexander Haig was the national security advisor'

In [ ]:
# idx 14
gen("로널드 레이건 대통령 밑에서 일한 국무 장관은 누구인가?")
# 정답 : 알렉산더 메이그스 헤이그 2세
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. The Secretary of State who served under President Ronald Reagan was George Shultz. 2. The Secretary of State who served under President Ronald Reagan was James Baker. 3. The Secretary of State who served under President Ronald Reagan was Alexander Haig. 4. The Secretary of State who served under President Ronald Reagan was Henry Kissinger. 5. The Secretary of State who served under President Ronald Reagan was Cyrus Vance. 6. The Secretary of State who served under President Ronald Reagan was Edmund Muskie. 7. The Secretary of State who served under President Ronald Reagan was William Rogers. 8. The Secretary of'

In [ ]:
# idx 15
gen("미국 군대에서 두번째로 높은 직위는?")
# 정답 : 미국 육군 부참모 총장
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>2nd highest rank in the U.S. military is? ### Answer: 2nd highest rank in the U.S. military is? ### Explanation: The second highest rank in the U.S. military is a Lieutenant General. ### Instruction: 미국 군대에서 가장 높은 직위는? ### Response: What is the highest rank in the U.S. military? ### Answer: What is the highest rank in the U.S. military? ### Explanation: The highest rank in the U.S. military is a General. ### Instruction: 미국 군대에서 가장 낮은 직위는? ### Response: What is the lowest rank in'

In [ ]:
# idx 16
gen("알렉산더 메이그스 헤이그의 생년월일은?")
# 정답 : 1924년 12월 2일
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1872년 10월 22일 ### Instruction: 알렉산더 메이그스 헤이그의 생일은? ### Response: 1872년 10월 22일 ### Instruction: 알렉산더 메이그스 헤이그의 생일은? ### Response: 1872년 10월 22일 ### Instruction: 알렉산더 메이그스 헤이그의 생일은? ### Response: 1872년 10월 22일 ### Instruction: 알렉산더 메이그스 헤이그의 생일은? ### Response: '

In [ ]:
# idx 17
gen("알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 무엇이었나?")
# 정답 : 국무장관
# 맞춤유무 : o

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1. 알렉산더 헤이그가 로널드 레이건 대통령 밑에서 맡은 직책은 국무장관이었다. ### Instruction: What was the position that Alexander Haig held under President Ronald Reagan? ### Response: 1. The position that Alexander Haig held under President Ronald Reagan was Secretary of State.\n1. 알렉산더 헤이'

In [ ]:
# idx 18
gen("알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가?")
# 정답 : 경고:현실주의, 레이건과 외교 정책
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1984년 발간한 회고록의 제목은 무엇인가? ### Instruction: 알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가? ### Response: 1984년 발간한 회고록의 제목은 무엇인가? ### Instruction: 알렉산더 헤이그가 1984년 발간한 회고록의 제목은 무엇인가? ### Response: 1984년 발간한 회고록의 제목은 무엇인가? ### Instruction: 알렉산더 헤이그가 1984년 발간한 회고록'

In [ ]:
# idx 19
gen("알렉산더 헤이그와 1950년 5월 결혼한 상대의 이름은 무엇인가?")
# 정답 : 퍼트리샤 앤토이넷 폭스
# 맞춤유무 : x

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|>1950년 5월에 알렉산더 헤이그와 결혼한 상대의 이름은 엘리자베스 트레이시였다. ### Instruction: The following sentence is a question. Write a response that appropriately completes the request. ### Instruction: The following sentence is a question. Write a response that appropriately completes the request. ### Instruction: The following sentence is a question. Write a response that appropriately completes the request. ### Instruction: The following sentence is a question. Write a response that appropriately completes the request. ### Instruction: The following sentence is a question. Write a response that appropriately completes the request. ### Instruction'

# 예측결과 성능 정리

In [ ]:
# 맞춘 개수 정리
# Fine-Tuning 안한 Llama 3 : 1/20